In [5]:
import pandas as pd
import numpy as np
from sklearn import * 
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [6]:
df = pd.read_csv("/data/kaggle/house-prices/data_combined_cleaned.csv")
del df["Id"]
df = df[~np.isnan(df.SalesPrice)]
df.head()


,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,LotConfig,LandSlope,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalesPrice
0,60,RL,65.0,8450,Pave,None,Reg,Lvl,Inside,Gtl,...,0,None,None,None,0,2,2008,WD,Normal,208500.0
1,20,RL,80.0,9600,Pave,None,Reg,Lvl,FR2,Gtl,...,0,None,None,None,0,5,2007,WD,Normal,181500.0
2,60,RL,68.0,11250,Pave,None,IR1,Lvl,Inside,Gtl,...,0,None,None,None,0,9,2008,WD,Normal,223500.0
3,70,RL,60.0,9550,Pave,None,IR1,Lvl,Corner,Gtl,...,0,None,None,None,0,2,2006,WD,Abnorml,140000.0
4,60,RL,84.0,14260,Pave,None,IR1,Lvl,FR2,Gtl,...,0,None,None,None,0,12,2008,WD,Normal,250000.0


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1460 entries, 0 to 1459
Data columns (total 79 columns):
MSSubClass       1460 non-null int64
MSZoning         1460 non-null object
LotFrontage      1460 non-null float64
LotArea          1460 non-null int64
Street           1460 non-null object
Alley            1460 non-null object
LotShape         1460 non-null object
LandContour      1460 non-null object
LotConfig        1460 non-null object
LandSlope        1460 non-null object
Neighborhood     1460 non-null object
Condition1       1460 non-null object
Condition2       1460 non-null object
BldgType         1460 non-null object
HouseStyle       1460 non-null object
OverallQual      1460 non-null int64
OverallCond      1460 non-null int64
YearBuilt        1460 non-null int64
YearRemodAdd     1460 non-null int64
RoofStyle        1460 non-null object
RoofMatl         1460 non-null object
Exterior1st      1460 non-null object
Exterior2nd      1460 non-null object
MasVnrType       1460 no

In [23]:
target = "SalesPrice"
y = np.log(df[target])
X = df.copy()
del X[target]
X = pd.get_dummies(X, drop_first=True)

X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, 
                                            test_size = 0.3, random_state = 123)
pipe = pipeline.Pipeline([
    ("poly", preprocessing.PolynomialFeatures(degree=1, include_bias=False)),
    ("scaler", preprocessing.StandardScaler()),
    ("est", linear_model.Lasso(alpha=0.01))
])

pipe.fit(X_train, y_train)
y_train_pred = pipe.predict(X_train)
y_test_pred = pipe.predict(X_test)

print("Training MSE", metrics.mean_squared_error(y_train, y_train_pred),
     "\nTesting MSE", metrics.mean_squared_error(y_test, y_test_pred),
     "\nTraining R2", metrics.r2_score(y_train, y_train_pred),
     "\nTesting R2", metrics.r2_score(y_test, y_test_pred))

Training MSE 0.018098827677061335 
Testing MSE 0.015542696024803503 
Training R2 0.8906252307539918 
Testing R2 0.892702031118294


In [24]:
target = "SalesPrice"
y = np.log(df[target])
X = df.copy()
del X[target]
X = pd.get_dummies(X, drop_first=True)

X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, 
                                            test_size = 0.3, random_state = 1230000)
pipe = pipeline.Pipeline([
    ("poly", preprocessing.PolynomialFeatures(degree=1, include_bias=False)),
    ("scaler", preprocessing.StandardScaler()),
    ("est", linear_model.Lasso(alpha=0.01))
])

pipe.fit(X_train, y_train)
y_train_pred = pipe.predict(X_train)
y_test_pred = pipe.predict(X_test)

print("Training MSE", metrics.mean_squared_error(y_train, y_train_pred),
     "\nTesting MSE", metrics.mean_squared_error(y_test, y_test_pred),
     "\nTraining R2", metrics.r2_score(y_train, y_train_pred),
     "\nTesting R2", metrics.r2_score(y_test, y_test_pred))

Training MSE 0.017954674283604467 
Testing MSE 0.016130297755959592 
Training R2 0.8859868499722388 
Testing R2 0.901591436127307


In [82]:
np.random.seed(1234)
np.random.randn(10)

array([ 0.47143516, -1.19097569,  1.43270697, -0.3126519 , -0.72058873,
        0.88716294,  0.85958841, -0.6365235 ,  0.01569637, -2.24268495])

In [48]:
scores = model_selection.cross_val_score(pipe, X, y, cv = 5)

In [49]:
np.mean(scores)

0.8549996959457898

In [99]:
pipe = pipeline.Pipeline([
    ("poly", preprocessing.PolynomialFeatures(degree=1, include_bias=False)),
    ("scaler", preprocessing.StandardScaler()),
    ("est", linear_model.ElasticNet(random_state=1, max_iter=5000, tol = 1e-6))
])

param_grid = {
    "est__alpha": 10 ** np.linspace(-1, 2, 10),
    "est__l1_ratio": np.linspace(0, 1, 10)
}

gsearch = model_selection.GridSearchCV(pipe, param_grid, cv = 5)
gsearch.fit(X_train, y_train)

est = gsearch.best_estimator_

y_train_pred = est.predict(X_train)
y_test_pred = est.predict(X_test)

print("Training MSE", metrics.mean_squared_error(y_train, y_train_pred),
     "\nTesting MSE", metrics.mean_squared_error(y_test, y_test_pred),
     "\nTraining R2", metrics.r2_score(y_train, y_train_pred),
     "\nTesting R2", metrics.r2_score(y_test, y_test_pred),
     "\nBest params: ", gsearch.best_params_)

/Users/abulbasar/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/abulbasar/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/abulbasar/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/abulbasar/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Obje

/Users/abulbasar/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/abulbasar/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/abulbasar/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/abulbasar/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Obje

Training MSE 0.01741578041296794 
Testing MSE 0.01935000346193165 
Training R2 0.8894088551142663 
Testing R2 0.881948487223877 
Best params:  {'est__alpha': 1.0, 'est__l1_ratio': 0.0}


/Users/abulbasar/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


In [91]:
gsearch.best_params_

{'est__alpha': 0.03593813663804626}

In [92]:
gsearch.best_score_

0.7798491421912578

In [93]:
gsearch.best_estimator_

Pipeline(memory=None,
     steps=[('poly', PolynomialFeatures(degree=1, include_bias=False, interaction_only=False)), ('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('est', Lasso(alpha=0.03593813663804626, copy_X=True, fit_intercept=True,
   max_iter=1000, normalize=False, positive=False, precompute=False,
   random_state=None, selection='cyclic', tol=0.0001, warm_start=False))])

In [87]:
gsearch = model_selection.GridSearchCV?

In [ ]:
gsearch = model_selection.GridSearchCV

In [88]:
np.linspace(-3, -1, 10)

array([-3.        , -2.77777778, -2.55555556, -2.33333333, -2.11111111,
       -1.88888889, -1.66666667, -1.44444444, -1.22222222, -1.        ])

In [85]:
10 ** np.linspace(-3, -1, 10)

array([0.001     , 0.0016681 , 0.00278256, 0.00464159, 0.00774264,
       0.0129155 , 0.02154435, 0.03593814, 0.05994843, 0.1       ])

In [100]:
est = gsearch.best_estimator_

In [101]:
import pickle

In [103]:
with open("housing_v1.model", "wb") as f:
    pickle.dump(est, f)

In [104]:
with open("housing_v1.model", "rb") as f:
    est2 = pickle.load(f)

In [107]:
y_train_pred = est2.predict(X_train)
y_test_pred = est2.predict(X_test)

print("Training MSE", metrics.mean_squared_error(y_train, y_train_pred),
     "\nTesting MSE", metrics.mean_squared_error(y_test, y_test_pred),
     "\nTraining R2", metrics.r2_score(y_train, y_train_pred),
     "\nTesting R2", metrics.r2_score(y_test, y_test_pred),
     "\nBest params: ", gsearch.best_params_)

Training MSE 0.01741578041296794 
Testing MSE 0.01935000346193165 
Training R2 0.8894088551142663 
Testing R2 0.881948487223877 
Best params:  {'est__alpha': 1.0, 'est__l1_ratio': 0.0}
